<a href="https://colab.research.google.com/github/mifm/examples/blob/master/autograd_existence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install py_wake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.7 MB/s eta 0:00:00


In [2]:
from py_wake.examples.data.iea37 import IEA37_WindTurbines
from py_wake.wind_turbines.power_ct_functions import PowerCtNDTabular
from py_wake.wind_turbines import WindTurbine
from py_wake.utils.gradients import autograd
import numpy as np
from py_wake.examples.data.hornsrev1 import Hornsrev1Site
from py_wake.literature.gaussian_models import Zong_PorteAgel_2020
from py_wake.deflection_models.jimenez import JimenezWakeDeflection
from py_wake.superposition_models import LinearSum
from py_wake.utils.layouts import rectangle


class ExistenceWindTurbine(WindTurbine):
    def __init__(self, name="Existence Turbine"):
        self.existence_values = np.array([0.0, 1.0])
        self.wsp_lst = np.linspace(0, 25, 50)

        # Extract data from the IEA37 wind turbine
        base_turbine = IEA37_WindTurbines()
        diameter = base_turbine.diameter()
        hub_height = base_turbine.hub_height()
        power = base_turbine.power(self.wsp_lst)
        ct = base_turbine.ct(self.wsp_lst)

        # Prepare 3D power and ct arrays
        power_3d = np.array([power * ex for ex in self.existence_values]).T
        ct_3d = np.array([ct * ex for ex in self.existence_values]).T

        # Define powerCtFunction with existence as a dimension
        powerCtFunction = PowerCtNDTabular(
            input_keys=['ws', 'existence'],
            value_lst=[self.wsp_lst, self.existence_values],
            power_arr=power_3d,
            power_unit='W',
            ct_arr=ct_3d,
            default_value_dict={'existence': 1.0}
        )

        # Initialize the wind turbine with the new powerCtFunction
        super().__init__(name, diameter, hub_height, powerCtFunction)


site = Hornsrev1Site()
wt = ExistenceWindTurbine()
wfm = Zong_PorteAgel_2020(site, wt, deflectionModel=JimenezWakeDeflection(), superpositionModel= LinearSum())
x,y = rectangle(3,3, wt.diameter()*5)

def get_aep(existence):
    return wfm.aep(x=x, y=y,yaw=[20,-10,1],tilt=0, existence=existence).sum()

existence = 0.5
aep = get_aep(existence)
print("AEP: ", aep)
print("Gradients: ", autograd(get_aep)(existence))


AEP:  23.852822735102162
Gradients:  46.97447594918426


In [3]:
existence = 0.2
aep = get_aep(existence)
print("AEP: ", aep)
print("Gradients: ", autograd(get_aep)(existence))


AEP:  9.623047411982478
Gradients:  47.85111779696202
